In [1]:
import requests
import io

from datetime import date, timedelta

url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
c.head(3)

,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2019-12-31,Afghanistan,0,0,0,0
1,2020-01-01,Afghanistan,0,0,0,0
2,2020-01-02,Afghanistan,0,0,0,0


In [2]:
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv")

In [3]:
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
#data_load = pd.read_csv(f"Pred\\predDf2020-03-28.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-03-31,France,48409.0,0.0,3460.0,0.0,0.0,0.0
0,2020-03-31,China,82463.0,0.0,3310.0,0.0,0.0,0.0
0,2020-03-31,Italy,106285.0,0.0,12378.0,0.0,0.0,0.0
0,2020-03-31,Spain,89410.0,0.0,8072.0,0.0,0.0,0.0
0,2020-03-31,United States,184381.0,0.0,4071.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")

In [6]:
frames = [follow_df1,follow_df2,follow_df3,follow_df4,follow_df5]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-04-01,France,48409.0,52128.0,3460.0,3523.0,-3719.0,-63.0
0,2020-04-01,China,82463.0,82295.0,3310.0,3310.0,168.0,0.0
0,2020-04-01,Italy,106285.0,105792.0,12378.0,12430.0,493.0,-52.0
0,2020-04-01,Spain,89410.0,94417.0,8072.0,8189.0,-5007.0,-117.0
0,2020-04-01,United States,184381.0,189618.0,4071.0,4079.0,-5237.0,-8.0


In [7]:
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")